In [1]:
pip install tensorflow-text

In [2]:
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [4]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [5]:
import shutil
shutil.copy('/content/drive/MyDrive/Colab Notebooks/Projects/Document Clustering /archive (2).zip', '/content/sample_data')

'/content/sample_data/archive (2).zip'

In [6]:
!unzip '/content/sample_data/archive (2).zip'

Archive:  /content/sample_data/archive (2).zip
replace emails.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [7]:
import pandas as pd
data = pd.read_csv('/content/emails.csv')

In [8]:
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [9]:
print(data['message'][1])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

In [10]:
import email
documents = {'message' : []}
for i in range(1000):
  transform_message = email.message_from_string(data['message'][i]).get_payload()
  documents['message'].append(transform_message)
documents = pd.DataFrame(documents)

In [13]:
documents.sample(10)

,message
580,\n---------------------- Forwarded by Phillip ...
319,"Johnnie,\n\nThank you for meeting with me on F..."
629,"John,\n\n Denver's short rockies position bey..."
181,"Lucy,\n\n Did you get set up on the checking a..."
299,"Ray,\n\nIs there any detail on the gas cost pr..."
665,"Greg,\n\n Got your message. Good luck on the ..."
331,---------------------- Forwarded by Phillip K ...
852,is this still an issue?
530,I can't open a winmail.dat file. can you send...
835,---------------------- Forwarded by Phillip K ...


In [13]:
import string
def preprocess(text):
    text = text.lower()
    no_punct=[words for words in text if words not in string.punctuation and words != '\n']
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

In [14]:
documents['message'] = documents['message'].map(lambda x : preprocess(x))

In [15]:
def get_document_embedding(documents):
  preprocessed_documents = bert_preprocess(documents)
  return bert_encoder(preprocessed_documents)['pooled_output']

In [20]:
temp = get_document_embedding(documents['message'][:100])

In [21]:
temp

<tf.Tensor: shape=(100, 768), dtype=float32, numpy=
array([[-0.9076078 , -0.38919416, -0.47276944, ..., -0.36286288,
        -0.66916466,  0.9240471 ],
       [-0.6266307 , -0.41573063, -0.8728297 , ..., -0.83001655,
        -0.5642164 ,  0.6867771 ],
       [-0.8215266 , -0.23296127,  0.24029484, ...,  0.21544626,
        -0.4997106 ,  0.8104935 ],
       ...,
       [-0.7180374 , -0.2805542 ,  0.2320496 , ...,  0.18522458,
        -0.55084   ,  0.6952226 ],
       [-0.23385939, -0.35295388, -0.576032  , ..., -0.26842844,
        -0.46508124,  0.61917526],
       [-0.78533477, -0.48900557, -0.80588245, ..., -0.64937145,
        -0.63769567,  0.8431077 ]], dtype=float32)>